# GPU functionality test

## Minimal `torch` example for 10.2 `cuda` error test

In [1]:
import torch
U_1=torch.randn(320, device="cuda")
U_2=torch.randn(13,320, device="cuda")
x=torch.randn(2,320,13,2, device="cuda")
lhs = torch.matmul(torch.matmul(x.permute(0, 3, 2, 1), U_1),
                     U_2)
torch.cuda.synchronize()

/gpfs/space/home/ondrejse/jupyter-base/ml-tartu-electricity-prediciton/mltartu-kernel/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Minimal `darts` model training with GPU example

In [2]:
import numpy as np
import pandas as pd

from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel
from darts.metrics import mape
from darts.datasets import AirPassengersDataset

In [3]:
# Read data:
series = AirPassengersDataset().load()
series = series.astype(np.float32)

# Create training and validation sets:
train, val = series.split_after(pd.Timestamp("19590101"))

# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

In [4]:
my_model = RNNModel(
    model="RNN",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=5,
    optimizer_kwargs={"lr": 1e-3},
    model_name="Air_RNN",
    log_tensorboard=True,
    random_state=42,
    training_length=20,
    input_chunk_length=14,
    force_reset=True,
    pl_trainer_kwargs={
        "accelerator": "gpu",
        "devices": [0],
    },
)

In [5]:
# Necessary for HPC instance of University of Tartu
# Old dependency uses SLURMS deprecated parameter
# This code sets it as it should be
import os

os.environ["SLURM_NTASKS_PER_NODE"] = "2"

In [6]:
my_model.fit(train_transformed, val_series=val_transformed)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | RNN              | 460   
4 | V             | Linear           | 21    
---------------------------------------------------
481       Trainable params
0         Non-trainable params
481       Total params
0.002     Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


/gpfs/space/home/ondrejse/jupyter-base/ml-tartu-electricity-prediciton/mltartu-kernel/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1558: PossibleUserWarning: The number of training batches (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:  88%|████████▊ | 7/8 [00:00<00:00,  9.34it/s, loss=0.0777, v_num=logs, train_loss=0.0391]
Validation: 0it [00:00, ?it/s]
Epoch 1:  88%|████████▊ | 7/8 [00:00<00:00, 76.83it/s, loss=0.0602, v_num=logs, train_loss=0.0115, val_loss=0.429]
Validation: 0it [00:00, ?it/s]
Epoch 2:  88%|████████▊ | 7/8 [00:00<00:00, 78.43it/s, loss=0.0517, v_num=logs, train_loss=0.0544, val_loss=0.303]
Validation: 0it [00:00, ?it/s]
Epoch 3:  88%|████████▊ | 7/8 [00:00<00:00, 82.05it/s, loss=0.0396, v_num=logs, train_loss=0.0403, val_loss=0.233]
Validation: 0it [00:00, ?it/s]
Epoch 4:  88%|████████▊ | 7/8 [00:00<00:00, 81.63it/s, loss=0.0369, v_num=logs, train_loss=0.040, val_loss=0.204] 
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 8/8 [00:00<00:00, 68.68it/s, loss=0.0369, v_num=logs, train_loss=0.040, val_loss=0.220]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 8/8 [00:00<00:00, 66.59it/s, loss=0.0369, v_num=logs, train_loss=0.040, val_loss=0.220]
